## Pytorch Tutorials : Optimizing
- [Pytorch Tutorials](https://pytorch.org/tutorials/)
- [Optimizing](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)

### Optimizing Model Parameters
모델에 대한 매개 변수를 최적화하여 모델을 교육, 검증 및 테스트할 때입니다. 모델을 훈련시키는 것은 반복적인 과정이다. 각 반복(에포크)에서 모델은 출력에 대해 추측하고, 추측(손실)의 오류를 계산하며, 매개 변수와 관련된 오류의 파생물을 수집하며, 경사 하강법을 사용하여 이러한 매개 변수를 최적화한다.

앞선 튜토리얼에서 진행한 코드 가져옴

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Hyperparameters
하이퍼 파라미터는 모델 최적화 프로세스를 제어할 수 있는 조정 가능한 파라미터입니다. 서로 다른 초 매개 변수 값이 모델 교육 및 수렴 속도에 영향을 미칠 수 있습니다.

- Number of Epochs : 데이터 집합에 대해 반복할 횟수
- Batch Size : 매개 변수가 업데이트되기 전에 네트워크를 통해 전파되는 데이터 샘플의 수
- Learning Rate : 각 배치/에포크에서 모델 매개 변수를 업데이트하는 양입니다. 값이 작을수록 학습 속도가 느려지는 반면 값이 클수록 훈련 중에 예측할 수 없는 동작이 발생할 수 있습니다.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization Loop
하이퍼 파라미터를 설정하면 최적화 루프를 통해 모델을 교육하고 최적화할 수 있습니다. 최적화 루프의 각 반복을 epoch라고 합니다.

- Train Loop : 교육 데이터 세트에 걸쳐 반복되고 최적의 매개 변수로 수렴되도록 시도합니다.
- Validation/Test Loop : 테스트 데이터 세트에 걸쳐 반복하여 모델 성능이 향상되고 있는지 확인합니다.

loss function  
일부 훈련 데이터가 제공되면 훈련되지 않은 네트워크는 정답을 제공하지 않을 수 있습니다. 손실 함수 는 얻어진 결과와 목표값의 차이 정도를 측정하는 것으로 훈련 시 최소화하고자 하는 손실 함수이다. 손실을 계산하기 위해 주어진 데이터 샘플의 입력을 사용하여 예측하고 실제 데이터 레이블 값과 비교합니다.

In [3]:
loss_fn = nn.CrossEntropyLoss()

optimizer  
최적화는 각 훈련 단계에서 모델 오류를 줄이기 위해 모델 매개변수를 조정하는 프로세스입니다. 최적화 알고리즘 은 이 프로세스가 수행되는 방식을 정의합니다. 모든 최적화 논리는 optimizer개체에 캡슐화됩니다.

- `optimizer.zero_grad()` : 모델 파라미터의 그래디언트를 재설정합니다. 기본적으로 그라데이션이 합산되며, 이중 계산을 방지하기 위해 각 반복 시 그라데이션이 명시적으로 0으로 설정됩니다.
- loss.backward() 호출을 통해 예측 손실을 역전파합니다. PyTorch는 각 파라미터와 함께 손실의 그래디언트를 축적합니다.
- `optimizer.step()` : 역전파에서 수집된 그레이디언트로 파라미터를 조정합니다.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Full Implementation  
train_lopp와 test_loop 정의한다.

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295083  [    0/60000]
loss: 2.287841  [ 6400/60000]
loss: 2.261963  [12800/60000]
loss: 2.259256  [19200/60000]
loss: 2.256648  [25600/60000]
loss: 2.207359  [32000/60000]
loss: 2.229725  [38400/60000]
loss: 2.181999  [44800/60000]
loss: 2.178864  [51200/60000]
loss: 2.157548  [57600/60000]
Test Error: 
 Accuracy: 34.7%, Avg loss: 2.143484 

Epoch 2
-------------------------------
loss: 2.145298  [    0/60000]
loss: 2.142932  [ 6400/60000]
loss: 2.074026  [12800/60000]
loss: 2.099154  [19200/60000]
loss: 2.058950  [25600/60000]
loss: 1.978877  [32000/60000]
loss: 2.018026  [38400/60000]
loss: 1.926352  [44800/60000]
loss: 1.934429  [51200/60000]
loss: 1.868205  [57600/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.857746 

Epoch 3
-------------------------------
loss: 1.882876  [    0/60000]
loss: 1.860648  [ 6400/60000]
loss: 1.729925  [12800/60000]
loss: 1.786824  [19200/60000]
loss: 1.679530  [25600/60000]
loss: 1.617365  [32000/600